In [9]:
import os
import rasterio as rio
from rasterio.merge import merge
from rasterio.enums import Resampling
from rasterio.warp import reproject, Resampling
from osgeo import gdal, gdal_array
import numpy as np

In [2]:
# Directory contenente i file tif
input_folder = r'C:\Users\giova\OneDrive\Desktop\prova new dataset'
output_file = r'C:\Users\giova\OneDrive\Desktop\prova new dataset\merged\3 Zeuli 220623 ORTHO_MULTI_11.tif'

# Lista per memorizzare i riferimenti a tutti i file tif
tif_files = []

In [3]:
# Scansione della cartella per individuare i file tif
for root, dirs, files in os.walk(input_folder):
    for file in files:
        if file.endswith('.tif'):
            tif_files.append(os.path.join(root, file))

tif_files

['C:\\Users\\giova\\OneDrive\\Desktop\\prova new dataset\\3 Zeuli 220623 ORTHO_Blue-444.tif',
 'C:\\Users\\giova\\OneDrive\\Desktop\\prova new dataset\\3 Zeuli 220623 ORTHO_Blue.tif',
 'C:\\Users\\giova\\OneDrive\\Desktop\\prova new dataset\\3 Zeuli 220623 ORTHO_Green-531.tif',
 'C:\\Users\\giova\\OneDrive\\Desktop\\prova new dataset\\3 Zeuli 220623 ORTHO_Green.tif',
 'C:\\Users\\giova\\OneDrive\\Desktop\\prova new dataset\\3 Zeuli 220623 ORTHO_NIR.tif',
 'C:\\Users\\giova\\OneDrive\\Desktop\\prova new dataset\\3 Zeuli 220623 ORTHO_Red edge-705.tif',
 'C:\\Users\\giova\\OneDrive\\Desktop\\prova new dataset\\3 Zeuli 220623 ORTHO_Red edge-740.tif',
 'C:\\Users\\giova\\OneDrive\\Desktop\\prova new dataset\\3 Zeuli 220623 ORTHO_Red edge.tif',
 'C:\\Users\\giova\\OneDrive\\Desktop\\prova new dataset\\3 Zeuli 220623 ORTHO_Red-650.tif',
 'C:\\Users\\giova\\OneDrive\\Desktop\\prova new dataset\\3 Zeuli 220623 ORTHO_Red.tif',
 'C:\\Users\\giova\\OneDrive\\Desktop\\prova new dataset\\3 Zeuli 220

In [4]:
# Lista per memorizzare i raster per ogni banda
band_rasters = []
# Apre e legge la prima banda di ogni tif
for tif_file in tif_files:
    with rasterio.open(tif_file) as src:
        band_rasters.append(src.read(1))

band_rasters

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 

In [5]:
print(f"height = {len(band_rasters[0])}")
print(f"width = {len(band_rasters[0][0])}")

height = 12109
width = 12455


In [11]:
# Legge e memorizza le bande di tutti i file tif
band_rasters_aligned = []

# Allinea le matrici dei raster utilizzando il riferimento del primo raster
aligned_rasters = []
first_raster = band_rasters[0]
for band in band_rasters[1:]:
    reprojected_band = np.zeros_like(first_raster)
    reproject(
        source=band,
        destination=reprojected_band,
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=src.transform,
        dst_crs=src.crs,
        resampling=Resampling.nearest
    )
    aligned_rasters.append(reprojected_band)

# Aggiunge il primo raster e gli altri raster allineati alla lista dei raster
band_rasters_aligned = [first_raster] + aligned_rasters

with rio.open(tif_files[0]) as src:
    profile = src.profile

with rasterio.open(output_file, 'w', **profile) as dst:
    for i, band_data in enumerate (band_rasters_aligned, start=1):
        dst.write(band_data)

print("Operazione completata. Il tif a 11 bande è stato salvato in:", output_file)

ValueError: Source shape (12109, 12455) is inconsistent with given indexes 2

In [65]:
for i, x in enumerate(band_rasters_aligned):
    print(f"Band {i}, shape {x.shape}")

Band 0, shape (12109, 12455)
Band 1, shape (12109, 12455)
Band 2, shape (12109, 12455)
Band 3, shape (12109, 12455)
Band 4, shape (12109, 12455)
Band 5, shape (12109, 12455)
Band 6, shape (12109, 12455)
Band 7, shape (12109, 12455)
Band 8, shape (12109, 12455)
Band 9, shape (12109, 12455)
Band 10, shape (12109, 12455)


In [86]:
input_ds = gdal.Open(tif_files[0])

width = input_ds.RasterXSize
height = input_ds.RasterYSize
num_bands = len(band_rasters_aligned)
data_type = input_ds.GetRasterBand(1).DataType

output_tif = r'C:\Users\giova\OneDrive\Desktop\prova new dataset\merged\3 Zeuli 220623 ORTHO_MULTI_11.tif'
driver = gdal.GetDriverByName('GTiff')
output_ds = driver.Create(output_tif, width, height, num_bands, data_type)

output_ds.SetGeoTransform(input_ds.GetGeoTransform())
output_ds.SetProjection(input_ds.GetProjection())

for i, band in enumerate(band_rasters_aligned):
    band_data = band
    output_ds.GetRasterBand(i).WriteArray(band_data)

# Chiude i dataset
input_ds = None
output_ds = None

print("Processo completato. Il nuovo raster è stato creato:", output_tif)

AttributeError: 'NoneType' object has no attribute 'WriteArray'

In [56]:
# Unisce le bande allineate
merged_raster, out_transform = merge(band_rasters_aligned, resampling=Resampling.nearest)

# Crea il profilo per il nuovo tif
profile = src.profile
profile.update(count=len(band_rasters_aligned), transform=out_transform)

# Salva il tif a 11 bande
output_file = r'C:\Users\giova\OneDrive\Desktop\prova new dataset\merged\3 Zeuli 220623 ORTHO_MULTI_11.tif'
with rasterio.open(output_file, 'w', **profile) as dst:
    dst.write(merged_raster)

print("Operazione completata. Il tif a 11 bande è stato salvato in:", output_file)

AttributeError: 'numpy.ndarray' object has no attribute 'profile'